# Lesson Walkthrough

This notebook walks through the material for each of the lesson topics, in order, to ensure they run. This notebook is *not* the "source" for the lesson material, e.g. through `pandoc` conversion. It is rather an environment to quickly sketch out and test ideas for lesson development.

## Introduction (to the lesson overall)

This lesson introduces MongoDB, a document-oriented database that departs from the relational nature of SQL. Because SQL is so well-established and prevalent as a database technology, the terms "relational database" and "SQL" are often synonymous. Furthermore, the term "NoSQL" has come to encompass a whole class of database technologies that are non-relational in nature. There are other "NoSQL" technologies such as key-value stores and column stores that differ from MongoDB's document/collection approach, but we will only cover MongoDB today.

MongoDB trades off the strictness of SQL for a degree of simplicity and flexibility often desirable for scientists who MongoDB trades off the strictness of SQL for a degree of simplicity and
flexibility often desirable for scientists who are often not certain about the
best schema design for new/changing data sets. Whereas SQL lends itself well to
enforcing a schema and thus ensuring data validation at the database level, a
system like MongoDB does not require setting/migrating schemas to get started
with data management. However, this means that application-level data
validation (e.g. before adding to the database) is particularly
important. MongoDB can pick up on implicit schema in your data through the
creation of indexes, which will speed up queries significantly for large data
sets.

This lesson was originally developed 
at the [Lawrence Berkeley National Laboratory](http://lbl.gov), where many scientists use MongoDB servers hosted by [NERSC](http://nersc.gov) to [manage workflows](https://pythonhosted.org/FireWorks/) and [process data](https://pythonhosted.org/pymatgen-db/) on supercomputing clusters. The example data for this lesson is from the [Materials Project](https://materialsproject.org), which hosts computed information for tens of thousands of known and predicted inorganic crystalline compounds.

## Prerequisites

## Getting Ready

## Introduction/Setup

I'd like at least two challenges per topic. A challenge should be a multiple-choice question (MCQ) or a Parson problem. Likely will be two MCQs. Come up with these first, then base the topic material on it.

## Connect to / Import into a Mock Database

To accommodate folks who cannot get a real database server running locally on their system, and also to demonstrate the utility of "mocking" for testing real access patterns without an external dependency, we will use the `mongomock` Python library to walk through MongoDB ideas for the first few topics. If you have a Mongo server running, feel free to use `pymongo.MongoClient` instead.

First, let's connect to the (mock) server an get a handle for our client.

In [207]:
import json
from mongomock import MongoClient
from pymongo import MongoClient

def reset_materials():
    client = MongoClient()
    db = client.swc
    db.materials.drop()
    with open('data/mongo-novice-materials.json') as f:
        db.materials.insert_many(json.load(f))

In [1]:
from mongomock import MongoClient
from pymongo import MongoClient

client = MongoClient()

A MongoDB instance can host multiple databases, which are created dynamically. Here, we will supply a database name as an attribute of the client object, which will prompt MongoDB to create the database with that name if it doesn't exists.

In [2]:
# Refer to the Software Carpentry ("swc") database
db = client.swc
print(db)

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'swc')


In Python, we can `print` something to look at it's string representation, which is often designed to be human-readable and can give us an idea of what's going on. We see here that what we're calling `db` is a Database with the name "swc" that we're accessing through a Mongo client connected to localhost on port 27017.

In [3]:
print(db.materials)

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'swc'), 'materials')


A MongoDB database is organized as a set of collections, each of which contains a set of documents. To first order, you can for now think of a collection as corresponding to a table in SQL and a collection document as corresponding to a table row in SQL.

Just as with databases themselves, database collections are created dynamically in MongoDB. Above, we created a `materials` collection in our database simply by referring to it by name.

Now, let's load data from a file and import it as documents into our collection:

In [4]:
import json

We first import the `json` module from the Python standard library. `JSON`, which stands for "Javascript Object Notation", is a way to express simple data structures that is widely used in web-based applications. We'll go over the format in the next topic when we construct a document to insert into our collection, but for now let's focus on importing data that we're given.

In [5]:
db.materials.drop()
with open('data/mongo-novice-materials.json') as f:
    db.materials.insert_many(json.load(f))

We are using a Python context manager to open a file and ensure that it is closed when we are done processing the file contents. In this case, we use the `json` module to load the file contents as Python-native data structures, which we then hand off to the `insert_many` method of database collections to insert all of the loaded documents.

In [6]:
db.materials.count()

66140

To confirm we have the data loaded, we use the `count()` method of a collection object and see that we have more than zero documents in our `materials` collection.

In [49]:
with open('data/mongo-novice-materials.json') as f:
    dataset = json.load(f)

In [50]:
print(type(dataset))
print(type(dataset[0]))

<class 'list'>
<class 'dict'>


In [51]:
db.publications.count()

0

## Insert Data

In [52]:
!wget "https://raw.githubusercontent.com/materialsproject/pymatgen/master/pymatgen/core/periodic_table.json" -O data/periodic_table.json

--2016-02-24 16:35:05--  https://raw.githubusercontent.com/materialsproject/pymatgen/master/pymatgen/core/periodic_table.json
Resolving raw.githubusercontent.com... 199.27.79.133
Connecting to raw.githubusercontent.com|199.27.79.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 120293 (117K) [text/plain]
Saving to: 'data/periodic_table.json'

data/periodic_table 100%[=====================>] 117.47K  --.-KB/s   in 0.1s   

2016-02-24 16:35:06 (853 KB/s) - 'data/periodic_table.json' saved [120293/120293]



In [53]:
import json
with open('data/periodic_table.json') as f:
    periodic_table = json.load(f)

In [54]:
from operator import itemgetter
def pluck(key, mappings):
    return map(itemgetter(key), mappings)

In [55]:
db.elements.insert_many([v for v in periodic_table.values()])

In [56]:
def project_only(*keys):
    doc = {"_id": 0}
    doc.update({k: 1 for k in keys})
    return doc

In [57]:
periodic_table['Cu']

{'Atomic mass': 63.546,
 'Atomic no': 29,
 'Atomic radius': 1.35,
 'Atomic radius calculated': 1.45,
 'Boiling point': '3200 K',
 'Brinell hardness': '874 MN m<sup>-2</sup>',
 'Bulk modulus': '140 GPa',
 'Coefficient of linear thermal expansion': '16.5 x10<sup>-6</sup>K<sup>-1</sup>',
 'Common oxidation states': [2],
 'Critical temperature': 'no data K',
 'Density of solid': '8920 kg m<sup>-3</sup>',
 'Electrical resistivity': '1.72 10<sup>-8</sup> &Omega; m',
 'Electronic structure': '[Ar].3d<sup>10</sup>.4s<sup>1</sup>',
 'Ionic radii': {'1': 0.91, '2': 0.87, '3': 0.68},
 'Liquid range': '1842.23 K',
 'Melting point': '1357.77 K',
 'Mendeleev no': 72,
 'Mineral hardness': '3.0',
 'Molar volume': '7.11 cm<sup>3</sup>',
 'Name': 'Copper',
 'Oxidation states': [1, 2, 3, 4],
 'Poissons ratio': '0.34',
 'Reflectivity': '90 %',
 'Refractive index': 'no data',
 'Rigidity modulus': '48 GPa',
 'Superconduction temperature': 'no data K',
 'Thermal conductivity': '400 W m<sup>-1</sup> K<sup>-1<

In [58]:
[d for d in db.elements.find({}, project_only('Name', 'Atomic no')).limit(3)]

[{'Atomic no': 12, 'Name': 'Magnesium'},
 {'Atomic no': 39, 'Name': 'Yttrium'},
 {'Atomic no': 23, 'Name': 'Vanadium'}]

In [59]:
[d for d in db.materials.find({}, project_only('material_id', 'pretty_formula', 'nelements')).limit(3)]

[{'material_id': 'mp-568345', 'nelements': 1, 'pretty_formula': 'Fe'},
 {'material_id': 'mp-12671', 'nelements': 3, 'pretty_formula': 'Er2SO2'},
 {'material_id': 'mp-1703', 'nelements': 2, 'pretty_formula': 'YbZn'}]

In [60]:
db.materials.find_one({"chemsys": "Na-O"})

{'_id': ObjectId('56ce4c367943f62692beb011'),
 'chemsys': 'Na-O',
 'elasticity': None,
 'elements': ['Na', 'O'],
 'material_id': 'mp-2340',
 'nelements': 2,
 'pretty_formula': 'Na2O2',
 'spacegroup': {'crystal_system': 'hexagonal',
  'hall': 'P -6 -2',
  'number': 189,
  'point_group': '-6m2',
  'source': 'spglib',
  'symbol': 'P-62m'}}

In [61]:
from datetime import datetime
material = {"fake": True,
            "elements": ["Na", "O"],
            "band_gap": 1.736,
            "last_updated": datetime.utcnow(),
            "spacegroup": {"crystal_system": "hexagonal", "number": 189}}

In [62]:
result = db.materials.insert_one(material)

In [63]:
result.inserted_id

ObjectId('56ce4c3b7943f62692bfb2c4')

In [64]:
print(result.inserted_id.generation_time)

2016-02-25 00:35:07+00:00


In [65]:
from bson import json_util
print(json_util.dumps(db.materials.find_one(result.inserted_id),
                      indent=2))

{
  "spacegroup": {
    "number": 189,
    "crystal_system": "hexagonal"
  },
  "elements": [
    "Na",
    "O"
  ],
  "_id": {
    "$oid": "56ce4c3b7943f62692bfb2c4"
  },
  "band_gap": 1.736,
  "fake": true,
  "last_updated": {
    "$date": 1456360507412
  }
}


In [66]:
result = db.materials.delete_many({"fake": True})

In [67]:
result.deleted_count

1

## Find data

You can use the `find()` method to issue a query to retrieve data from a collection in MongoDB. All queries in MongoDB have the scope of a single collection.

Queries can return all documents in a collection or only the documents that match a specified filter or criteria. You can specify the filter or criteria in a document and pass it as a parameter to the `find()` method.

The `find()` method returns query results in a cursor, which is an iterable object that yields documents.

### Gotta catch 'em all

To return all documents in a collection, call the `find()` method without a criteria document. 

In [68]:
cursor = db.materials.find()

Let's iterate over the cursor and print a few material ids.

In [69]:
how_many = 5
counter = 0
for document in cursor:
    if counter < how_many:
        print(document['material_id'])
        counter += 1
    else:
        break

mp-568345
mp-12671
mp-1703
mp-5152
mp-569624


There's an easier way to limit how many documents are yielded by a cursor:

In [70]:
for document in cursor.limit(5):
    print(document['material_id'])

mp-552787
mp-188
mp-600216
mp-2310
mp-780541


### Query by a top-level field

The following operation finds documents whose **nelements** field equals **3**:

In [71]:
cursor = db.materials.find({"nelements": 3})

Let's print ("pretty print", for nice indentation) a few of the results:

In [72]:
from pprint import pprint

for doc in cursor.limit(3):
    pprint(doc)

{'_id': ObjectId('56ce4c367943f62692beb002'),
 'chemsys': 'Er-O-S',
 'elasticity': None,
 'elements': ['Er', 'O', 'S'],
 'material_id': 'mp-12671',
 'nelements': 3,
 'pretty_formula': 'Er2SO2',
 'spacegroup': {'crystal_system': 'trigonal',
                'hall': '-P 3 2=',
                'number': 164,
                'point_group': '-3m',
                'source': 'spglib',
                'symbol': 'P-3m1'}}
{'_id': ObjectId('56ce4c367943f62692beb004'),
 'chemsys': 'La-O-Si',
 'elasticity': None,
 'elements': ['La', 'O', 'Si'],
 'material_id': 'mp-5152',
 'nelements': 3,
 'pretty_formula': 'La2SiO5',
 'spacegroup': {'crystal_system': 'monoclinic',
                'hall': '-P 2yab',
                'number': 14,
                'point_group': '2/m',
                'source': 'spglib',
                'symbol': 'P2_1/c'}}
{'_id': ObjectId('56ce4c367943f62692beb007'),
 'chemsys': 'Cl-Fe-O',
 'elasticity': {'G_Reuss': 4.800058831100799,
                'G_VRH': 15.695815587428928,
    

### Projection to select fields

That last query returned all fields for each document. We can use a projection, specified as JSON, to indicate which fields we want. The `_id` field is included by default -- we must be explicit if we don't want it returned.

In [73]:
cursor = db.materials.find({"nelements": 3},
                           {"material_id": 1, "pretty_formula": 1, "_id": 0})

for doc in cursor.limit(3):
    pprint(doc)

{'material_id': 'mp-12671', 'pretty_formula': 'Er2SO2'}
{'material_id': 'mp-5152', 'pretty_formula': 'La2SiO5'}
{'material_id': 'mp-552787', 'pretty_formula': 'FeClO'}


### Query by a field in an embedded document

To specify a condition on a field within an embedded document, use dot notation. Dot notation requires quotes around the whole dotted field name.

In [74]:
cursor = db.materials.find({"spacegroup.crystal_system": "cubic"})

print(cursor.count())

9408


Projection can take advantage of the same dot notation:

In [75]:
cursor = db.materials.find({"nelements": 2}, {"spacegroup.crystal_system": 1, "elements": 1, "_id": 0})

for doc in cursor.limit(3):
    pprint(doc)

{'elements': ['Yb', 'Zn'], 'spacegroup': {'crystal_system': 'cubic'}}
{'elements': ['Cr', 'Hf'], 'spacegroup': {'crystal_system': 'hexagonal'}}
{'elements': ['B', 'Lu'], 'spacegroup': {'crystal_system': 'cubic'}}


### Query by a field in an array

How many materials in our collection contain iron? When a field is an array, testing for membership has the same form as testing for equality:

In [76]:
db.materials.find({"elements": "Fe"}).count()

5813

If you supply an array as the value under test, we can see that four polymorphs of iron are present in our collection:

In [77]:
db.materials.find({"elements": ["Fe"]}).count()

4

### Challenge: Dot Notation and Projections

Which query below yields documents containing the crystal system and spacegroup number for all binary compounds?

In [78]:
cursor = db.materials.find({"nelements": 2}, {"spacegroup": {"crystal_system": 1, "number": 1}})
for doc in cursor.limit(3):
    pprint(doc)

ValueError: Unsupported projection option: number

In [79]:
cursor = db.materials.find({"nelements": 2}, {"spacegroup.crystal_system": 1, "spacegroup.number": 1})
for doc in cursor.limit(3):
    pprint(doc)

{'_id': ObjectId('56ce4c367943f62692beb003'),
 'spacegroup': {'crystal_system': 'cubic', 'number': 221}}
{'_id': ObjectId('56ce4c367943f62692beb005'),
 'spacegroup': {'crystal_system': 'hexagonal', 'number': 194}}
{'_id': ObjectId('56ce4c367943f62692beb006'),
 'spacegroup': {'crystal_system': 'cubic', 'number': 221}}


### Challenge: Combining Conditions

Which query below returns the number of binary oxides (oxygen-containing, two-element materials) in our collection?

In [80]:
db.materials.find({"elements": "O"}).limit(2).count()

38125

In [81]:
db.materials.find({"elements": "O", "nelements": 2}).count()

1519

In [82]:
db.materials.find({"elements": ["O"], "nelements": 2}).count()

0

## Specify Conditions with Operators

In [89]:
COMMON_PROJ = {
    "material_id": 1,
    "pretty_formula": 1,
    "_id": 0,
}

def print_a_few_for(query):
    proj = {k: 1 for k in query}
    proj.update(COMMON_PROJ)
    for doc in db.materials.find(query, proj).limit(5):
        pprint(doc)

query = {"nelements": {"$gte": 3}}
print_a_few_for(query)

{'material_id': 'mp-12671', 'nelements': 3, 'pretty_formula': 'Er2SO2'}
{'material_id': 'mp-5152', 'nelements': 3, 'pretty_formula': 'La2SiO5'}
{'material_id': 'mp-552787', 'nelements': 3, 'pretty_formula': 'FeClO'}
{'material_id': 'mp-780541', 'nelements': 3, 'pretty_formula': 'Fe4O7F'}
{'material_id': 'mp-31899', 'nelements': 4, 'pretty_formula': 'Ba2MnReO6'}


In [90]:
print_a_few_for({"nelements": {"$lt": 3}})

{'material_id': 'mp-568345', 'nelements': 1, 'pretty_formula': 'Fe'}
{'material_id': 'mp-1703', 'nelements': 2, 'pretty_formula': 'YbZn'}
{'material_id': 'mp-569624', 'nelements': 2, 'pretty_formula': 'HfCr2'}
{'material_id': 'mp-12660', 'nelements': 2, 'pretty_formula': 'LuB6'}
{'material_id': 'mp-188', 'nelements': 2, 'pretty_formula': 'AlPt3'}


In [93]:
db.materials.find({"chemsys": "Fe-O", "spacegroup.crystal_system": "cubic"}).count()

7

In [95]:
db.materials.find({
    "$or": [{"nelements": 2}, {"nelements": 4}]
}).count()

27370

In [98]:
print_a_few_for({"elasticity": {"$exists": True}})

{'elasticity': None, 'material_id': 'mp-568345', 'pretty_formula': 'Fe'}
{'elasticity': None, 'material_id': 'mp-12671', 'pretty_formula': 'Er2SO2'}
{'elasticity': None, 'material_id': 'mp-1703', 'pretty_formula': 'YbZn'}
{'elasticity': None, 'material_id': 'mp-5152', 'pretty_formula': 'La2SiO5'}
{'elasticity': {'G_Reuss': 64.86603926197118,
                'G_VRH': 66.1852057595004,
                'G_Voigt': 67.50437225702963,
                'K_Reuss': 189.54199905990907,
                'K_VRH': 189.5741939942109,
                'K_Voigt': 189.60638892851273,
                'calculations': {'energy_cutoff': 700.0,
                                 'kpoint_density': 7000,
                                 'pseudopotentials': ['Hf_pv', 'Cr_pv']},
                'elastic_anisotropy': 0.2037075326032598,
                'elastic_tensor': [[295.8188907773902,
                                    133.82065045871653,
                                    140.1170929005653,
                 

In [99]:
print_a_few_for({"elasticity": {"$ne": None}})

{'elasticity': {'G_Reuss': 64.86603926197118,
                'G_VRH': 66.1852057595004,
                'G_Voigt': 67.50437225702963,
                'K_Reuss': 189.54199905990907,
                'K_VRH': 189.5741939942109,
                'K_Voigt': 189.60638892851273,
                'calculations': {'energy_cutoff': 700.0,
                                 'kpoint_density': 7000,
                                 'pseudopotentials': ['Hf_pv', 'Cr_pv']},
                'elastic_anisotropy': 0.2037075326032598,
                'elastic_tensor': [[295.8188907773902,
                                    133.82065045871653,
                                    140.1170929005653,
                                    0.0,
                                    0.0,
                                    -0.036991826666666644],
                                   [133.82065045871653,
                                    299.3255081549423,
                                    141.6470726644416,
       

## Sorting

In [101]:
import pymongo
cursor = db.materials.find().sort([
    ("borough", pymongo.ASCENDING),
    ("address.zipcode", pymongo.DESCENDING)
])

In [194]:
COMMON_PROJ = {
    "material_id": 1,
    "pretty_formula": 1,
    "spacegroup.number": 1,
    "_id": 0,
}

def cursor_with_minimal_projection(query):
    proj = {k: 1 for k in query}
    proj.update(COMMON_PROJ)
    cursor = db.materials.find(query, proj)
    return cursor

def print_a_few_for(query):
    cursor = cursor_with_minimal_projection(query)
    for doc in cursor.limit(5):
        pprint(doc)

In [198]:
filt = {"elasticity": {"$ne": None}}
proj = {"elasticity.poisson_ratio": 1}
proj.update(COMMON_PROJ)
cursor = db.materials.find(filt, proj).sort("elasticity.poisson_ratio")
for doc in cursor.limit(5):
    pprint(doc)

{'elasticity': {'poisson_ratio': -0.07595596751510682},
 'material_id': 'mp-771798',
 'pretty_formula': 'WO3',
 'spacegroup': {'number': 204}}
{'elasticity': {'poisson_ratio': 0.042582069532848744},
 'material_id': 'mp-87',
 'pretty_formula': 'Be',
 'spacegroup': {'number': 194}}
{'elasticity': {'poisson_ratio': 0.05989488523534276},
 'material_id': 'mp-765892',
 'pretty_formula': 'MnCoO4',
 'spacegroup': {'number': 10}}
{'elasticity': {'poisson_ratio': 0.07135395365323711},
 'material_id': 'mp-611426',
 'pretty_formula': 'C',
 'spacegroup': {'number': 194}}
{'elasticity': {'poisson_ratio': 0.07391837849830045},
 'material_id': 'mp-23703',
 'pretty_formula': 'LiH',
 'spacegroup': {'number': 225}}


In [204]:
cursor = db.materials.find(filt, proj).sort("elasticity.poisson_ratio", -1)
for doc in cursor.limit(5):
    pprint(doc)

{'elasticity': {'poisson_ratio': 0.46752282089107655},
 'material_id': 'mp-1387',
 'pretty_formula': 'AlV3',
 'spacegroup': {'number': 223}}
{'elasticity': {'poisson_ratio': 0.46347893160099135},
 'material_id': 'mp-361',
 'pretty_formula': 'Cu2O',
 'spacegroup': {'number': 224}}
{'elasticity': {'poisson_ratio': 0.4622062265482987},
 'material_id': 'mp-544',
 'pretty_formula': 'Ti3Ir',
 'spacegroup': {'number': 223}}
{'elasticity': {'poisson_ratio': 0.4461861025154294},
 'material_id': 'mp-22060',
 'pretty_formula': 'Nb3In',
 'spacegroup': {'number': 223}}
{'elasticity': {'poisson_ratio': 0.4450687838211702},
 'material_id': 'mp-75',
 'pretty_formula': 'Nb',
 'spacegroup': {'number': 229}}


In [206]:
cursor = db.materials.find(filt, proj).sort([
        ("nelements", -1),
        ("elasticity.poisson_ratio", -1),
    ])
for doc in cursor.limit(5):
    pprint(doc)

{'elasticity': {'poisson_ratio': 0.3419981312210572},
 'material_id': 'mp-556194',
 'pretty_formula': 'SrSbSe2F',
 'spacegroup': {'number': 129}}
{'elasticity': {'poisson_ratio': 0.34027796672596883},
 'material_id': 'mp-12532',
 'pretty_formula': 'KAg2PS4',
 'spacegroup': {'number': 121}}
{'elasticity': {'poisson_ratio': 0.3349886462699885},
 'material_id': 'mp-557862',
 'pretty_formula': 'BaAg2(HgO2)2',
 'spacegroup': {'number': 125}}
{'elasticity': {'poisson_ratio': 0.3050725032443602},
 'material_id': 'mp-11806',
 'pretty_formula': 'LiMgSnPt',
 'spacegroup': {'number': 216}}
{'elasticity': {'poisson_ratio': 0.3042911533524732},
 'material_id': 'mp-546011',
 'pretty_formula': 'YZnAsO',
 'spacegroup': {'number': 129}}


In [189]:
cursor = db.materials.find({
        "nelements": {"$lt": 3}
    }, {
        "_id": 0,
        "nelements": 1,
        "elasticity.K_VRH": 1,
        "pretty_formula": 1,
        "spacegroup.crystal_system": 1,
        "material_id": 1,
    })

cursor = cursor.sort([("nelements", 1),("elasticity.K_VRH", -1)])

for doc in cursor.limit(5):
    pprint(doc)

{'elasticity': {'K_VRH': 435.66148729813784},
 'material_id': 'mp-611426',
 'nelements': 1,
 'pretty_formula': 'C',
 'spacegroup': {'crystal_system': 'hexagonal'}}
{'elasticity': {'K_VRH': 401.3286154019227},
 'material_id': 'mp-49',
 'nelements': 1,
 'pretty_formula': 'Os',
 'spacegroup': {'crystal_system': 'hexagonal'}}
{'elasticity': {'K_VRH': 365.08592365295135},
 'material_id': 'mp-8',
 'nelements': 1,
 'pretty_formula': 'Re',
 'spacegroup': {'crystal_system': 'hexagonal'}}
{'elasticity': {'K_VRH': 346.3227612888041},
 'material_id': 'mp-101',
 'nelements': 1,
 'pretty_formula': 'Ir',
 'spacegroup': {'crystal_system': 'cubic'}}
{'elasticity': {'K_VRH': 307.5241654938898},
 'material_id': 'mp-33',
 'nelements': 1,
 'pretty_formula': 'Ru',
 'spacegroup': {'crystal_system': 'hexagonal'}}


In [188]:
cursor = db.materials.find({
        "nelements": {"$lt": 3}
    }, {
        "_id": 0,
        "nelements": 1,
        "elasticity.K_VRH": 1,
        "pretty_formula": 1,
        "spacegroup.crystal_system": 1,
        "material_id": 1,
    }, sort=[
        ("nelements", 1),
        ("elasticity.K_VRH", -1)
    ])

for doc in cursor.limit(5):
    pprint(doc)

{'elasticity': {'K_VRH': 435.66148729813784},
 'material_id': 'mp-611426',
 'nelements': 1,
 'pretty_formula': 'C',
 'spacegroup': {'crystal_system': 'hexagonal'}}
{'elasticity': {'K_VRH': 401.3286154019227},
 'material_id': 'mp-49',
 'nelements': 1,
 'pretty_formula': 'Os',
 'spacegroup': {'crystal_system': 'hexagonal'}}
{'elasticity': {'K_VRH': 365.08592365295135},
 'material_id': 'mp-8',
 'nelements': 1,
 'pretty_formula': 'Re',
 'spacegroup': {'crystal_system': 'hexagonal'}}
{'elasticity': {'K_VRH': 346.3227612888041},
 'material_id': 'mp-101',
 'nelements': 1,
 'pretty_formula': 'Ir',
 'spacegroup': {'crystal_system': 'cubic'}}
{'elasticity': {'K_VRH': 307.5241654938898},
 'material_id': 'mp-33',
 'nelements': 1,
 'pretty_formula': 'Ru',
 'spacegroup': {'crystal_system': 'hexagonal'}}


In [22]:
systems = filter(None, db.materials.distinct("spacegroup.crystal_system"))
for s in systems:
    materials = list(db.materials.find(
        {"spacegroup.crystal_system": s}, {"spacegroup.number": 1}
    ).sort("spacegroup.number"))
    print(s, materials[0]["spacegroup"]["number"], materials[-1]["spacegroup"]["number"])
    

cubic 195 230
trigonal 143 167
monoclinic 3 15
hexagonal 169 194
orthorhombic 16 74
tetragonal 75 142
triclinic 1 2


## Update

In [35]:
import json

db.elements.drop()
with open('data/periodic_table.json') as f:
    for symbol, rest in json.load(f).items():
        doc = {"Symbol": symbol}
        doc.update(rest)
        db.elements.insert_one(doc)

In [43]:
halogens = ["F", "Cl", "Br", "I", "At"]
def halide_systems():
    for halogen in halogens:
        X = db.elements.find_one({"Symbol": halogen})["X"]
        for doc in db.elements.find({"X": {"$lt": X}}):
            yield "-".join(sorted([doc["Symbol"], halogen]))

In [46]:
[s for s in list(halide_systems()) if 'Cu' in s]

['Cu-F', 'Cl-Cu', 'Br-Cu', 'Cu-I', 'At-Cu']

In [117]:
reset_materials()

In [90]:
db.materials.find({"chemsys": "Cu-F"}).count()

2

In [91]:
result = db.materials.update_one(
    {"chemsys": "Cu-F"},
    {
        "$set": {
            "tags": ["halide"]
        },
        "$currentDate": {"lastModified": True}
    }
)

In [92]:
result.matched_count

1

In [93]:
result.modified_count

1

In [94]:
db.materials.find_one({"chemsys": "Cu-F"},
                      {"tags": 1, "lastModified": 1, "material_id": 1})

{'_id': ObjectId('56cedcde7943f6405b34a56e'),
 'lastModified': datetime.datetime(2016, 2, 25, 10, 52, 24, 303000),
 'material_id': 'mp-1229',
 'tags': ['halide']}

In [95]:
result = db.materials.update_one(
    {"material_id": "mp-1229"},
    {"$set": {"elasticity.calculations.source": "Private communication"}}
)

print(result.matched_count, result.modified_count)

1 1


In [106]:
result = db.materials.update_many(
    {"chemsys": {"$in": list(halide_systems())}},
    {"$addToSet": {"tags": "halide"}}
)

In [107]:
result.matched_count, result.modified_count

(1127, 0)

In [108]:
db.materials.find({"tags": "halide"}).count()

1127

In [110]:
result = db.materials.update_one(
    {"material_id": "mp-NaN"},
    {"$set": {"elasticity.calculations.source": "Private communication"}}
)

In [111]:
result.matched_count, result.modified_count

(0, 0)

In [130]:
result = db.materials.update_one(
    {"material_id": "mp-NaN"},
    {"$set": {"elasticity.calculations.source": "Private communication"}},
    upsert=True
)

In [131]:
result.matched_count, result.modified_count, result.upserted_id

(0, 0, ObjectId('56cf4065c46979cbe90cc42a'))

In [132]:
db.materials.find_one({"material_id": "mp-NaN"})

{'_id': ObjectId('56cf4065c46979cbe90cc42a'),
 'elasticity': {'calculations': {'source': 'Private communication'}},
 'material_id': 'mp-NaN'}

### Removing

In [133]:
result = db.materials.delete_many({"material_id": "mp-NaN"})

In [139]:
result = db.materials.delete_many({"nelements": 1})

In [140]:
result.deleted_count

419

In [141]:
result = db.materials.delete_many({})

In [142]:
result.deleted_count

65721

In [143]:
db.materials.count()

0

In [144]:
db.materials.drop()

### Re(Importing)

In [149]:
import json
from mongomock import MongoClient
#from pymongo import MongoClient

def reset_materials():
    client = MongoClient()
    db = client.swc
    db.materials.drop()
    with open('data/mongo-novice-materials.json') as f:
        db.materials.insert_many(json.load(f))

In [153]:
%timeit reset_materials()

1 loop, best of 3: 3.03 s per loop


In [163]:
with open('data/materials.mongoimportable.json', 'w') as outfile:
    with open('data/mongo-novice-materials.json') as infile:
        docs = json.load(infile)
        outfile.writelines([json.dumps(doc)+'\n' for doc in docs])

In [165]:
db.materials.drop()
!mongoimport --db swc --collection materials < data/materials.mongoimportable.json

connected to: 127.0.0.1
2016-02-25T10:46:31.004-0800 			53700	17900/second
2016-02-25T10:46:31.600-0800 check 9 66140
2016-02-25T10:46:31.607-0800 imported 66140 objects


### Aggregation

In [167]:
cursor = db.materials.aggregate(
    [
        {"$group": {"_id": "$nelements", "count": {"$sum": 1}}}
    ]
)

In [168]:
for doc in cursor:
    print(doc)

{'_id': 8, 'count': 5}
{'_id': 7, 'count': 68}
{'_id': 5, 'count': 5890}
{'_id': 4, 'count': 17561}
{'_id': 6, 'count': 866}
{'_id': 2, 'count': 9809}
{'_id': 3, 'count': 31522}
{'_id': 1, 'count': 419}


In [170]:
cursor = db.materials.aggregate(
    [
        {"$match": {"elements": "O"}},
        {"$group": {"_id": "$nelements", "count": {"$sum": 1}}}
    ]
)

for doc in cursor:
    print(doc)

{'_id': 8, 'count': 5}
{'_id': 1, 'count': 6}
{'_id': 7, 'count': 65}
{'_id': 5, 'count': 5484}
{'_id': 4, 'count': 15113}
{'_id': 6, 'count': 816}
{'_id': 2, 'count': 1519}
{'_id': 3, 'count': 15117}


In [173]:
cursor = db.materials.aggregate(
    [
        {"$match": {"elements": "O"}},
        {"$group": {"_id": "$nelements", "count": {"$sum": 1}}},
        {"$sort": {"count": -1}}
    ]
)

for doc in cursor:
    print(doc)

{'_id': 3, 'count': 15117}
{'_id': 4, 'count': 15113}
{'_id': 5, 'count': 5484}
{'_id': 2, 'count': 1519}
{'_id': 6, 'count': 816}
{'_id': 7, 'count': 65}
{'_id': 1, 'count': 6}
{'_id': 8, 'count': 5}


In [180]:
db.materials.find_one()

{'_id': ObjectId('56cf4c04c46979cbe913d4af'),
 'chemsys': 'Fe',
 'elasticity': None,
 'elements': ['Fe'],
 'material_id': 'mp-568345',
 'nelements': 1,
 'pretty_formula': 'Fe',
 'spacegroup': {'crystal_system': 'cubic',
  'hall': '-P 4 2 3',
  'number': 221,
  'point_group': '4/mmm',
  'source': 'spglib',
  'symbol': 'Pm-3m'}}

In [181]:
cursor = db.materials.aggregate(
    [
        {"$group": {
                "_id": "$spacegroup.number",
                "count": {"$sum": 1},
                "symbol": {"$first": "$spacegroup.symbol"},
            }},
        {"$sort": {"count": -1}},
        {"$limit": 10}
    ]
)

for doc in cursor:
    print(doc)

{'_id': 14, 'count': 5962, 'symbol': 'P2_1/c'}
{'_id': 1, 'count': 4593, 'symbol': 'P1'}
{'_id': 225, 'count': 4109, 'symbol': 'Fm-3m'}
{'_id': 2, 'count': 4010, 'symbol': 'P-1'}
{'_id': 62, 'count': 3562, 'symbol': 'Pmnb'}
{'_id': 15, 'count': 2736, 'symbol': 'C2/c'}
{'_id': 12, 'count': 2344, 'symbol': 'C2/m'}
{'_id': 139, 'count': 1797, 'symbol': 'I4/mmm'}
{'_id': 194, 'count': 1685, 'symbol': 'P6_3/mmc'}
{'_id': 221, 'count': 1630, 'symbol': 'Pm-3m'}


In [203]:
def get_chemistry(doc):
    anion = db.elements.find(
        {"Symbol": {"$in": doc["elements"]}},
        {"Symbol": 1, "X": 1, "_id": 0}
    ).sort([('X', -1)])[0]["Symbol"]
    if anion == "O":
        return "Oxide"
    elif anion == "S":
        return "Sulfide"
    elif anion == "F":
        return "Fluoride"
    elif anion == "Cl":
        return "Chloride"
    return None

In [204]:
for doc in db.materials.find({"elements": {"$in": ["O", "S", "F", "Cl"]}}, {"elements": 1}):
    anion = get_chemistry(doc)
    db.materials.update_one({"_id": doc["_id"]}, {"$set": {"anion_chemistry": anion}})

In [200]:
cursor = db.materials.aggregate(
    [
        {"$match": {"anion_chemistry": "Oxide"}},
        {"$group": {
                "_id": "$spacegroup.number",
                "count": {"$sum": 1},
                "symbol": {"$first": "$spacegroup.symbol"},
            }},
        {"$sort": {"count": -1}},
        {"$limit": 10}
    ]
)

for doc in cursor:
    print(doc)

{'_id': 14, 'count': 4013, 'symbol': 'P2_1/c'}
{'_id': 1, 'count': 3424, 'symbol': 'P1'}
{'_id': 2, 'count': 3039, 'symbol': 'P-1'}
{'_id': 15, 'count': 1856, 'symbol': 'C2/c'}
{'_id': 62, 'count': 1702, 'symbol': 'Pmnb'}
{'_id': 12, 'count': 1415, 'symbol': 'C2/m'}
{'_id': 8, 'count': 975, 'symbol': 'Cm'}
{'_id': 11, 'count': 877, 'symbol': 'P2_1/m'}
{'_id': 4, 'count': 759, 'symbol': 'P2_1'}
{'_id': 166, 'count': 606, 'symbol': 'R-3m'}


### Indexes

In [293]:
%%timeit

db.materials.distinct("spacegroup.crystal_system")

10 loops, best of 3: 47.1 ms per loop


In [294]:
db.materials.create_index([("spacegroup.crystal_system", 1)])

'spacegroup.crystal_system_1'

In [299]:
pprint(db.materials.index_information())

{'_id_': {'key': [('_id', 1)], 'ns': 'swc.materials', 'v': 1},
 'spacegroup.crystal_system_1': {'key': [('spacegroup.crystal_system', 1)],
                                 'ns': 'swc.materials',
                                 'v': 1}}


In [301]:
%%timeit

db.materials.distinct("spacegroup.crystal_system")

1000 loops, best of 3: 296 µs per loop


In [304]:
%timeit db.materials.find_one({"material_id": "mp-49"}, {"pretty_formula": 1, "_id": 0})

100 loops, best of 3: 12.1 ms per loop


In [306]:
db.materials.create_index([("material_id", 1), ("pretty_formula", 1)])

'material_id_1_pretty_formula_1'

In [307]:
%timeit db.materials.find_one({"material_id": "mp-49"}, {"pretty_formula": 1, "_id": 0})

1000 loops, best of 3: 187 µs per loop


In [308]:
db.materials.drop_index([("material_id", 1), ("pretty_formula", 1)])
db.materials.create_index([("pretty_formula", 1), ("material_id", 1)])

'pretty_formula_1_material_id_1'

In [309]:
%timeit db.materials.find_one({"material_id": "mp-49"}, {"pretty_formula": 1, "_id": 0})

100 loops, best of 3: 12.1 ms per loop
